# Application Definitions

## Overview

[NeXus Application Definitions](https://manual.nexusformat.org/classes/applications/index.html) define mandatory and optional class contents for specific applications.
ScippNexus' approach to application definitions is to consider them as a guide, without performing full validation.
This is to avoid getting in the way of the library user, e.g., when working with incomplete or partially broken files.
For example, ScippNexus will generally not validate that the tree structure conforms to a given application definition.

<div class="alert alert-warning">
    <b>Warning:</b>

ScippNexus' support for application definitions is currently experimental and the API is still subject to changes.

</div>

## Writing files

Skip ahead to [Reading files](#Reading-files) if you simply want to customize how data is read from existing files.
ScippNexus provides a customization point for writing content to NeXus files with `__setitem__`.
The requirements are that the value

1. provides an `nx_class` attribute that returns a valid NeXus class name such as `'NXdata'` or `scippnexus.NXdata` and
2. defines the `__write_to_nexus_group__` method that takes a `scippnexus.NXobject`, i.e., an open NeXus group, as its single argument.

`__write_to_nexus_group__` may then write its content to this group.
This can (and should) make use of ScippNexus features for writing Nexus fields (HDF5 datasets), such as automatic writing of the `units` attribute, or writing `datetime64` data.
Consider the following example:

In [ ]:
import scipp as sc
import scippnexus as snx


class MyData:
    nx_class = snx.NXdata  # required

    def __init__(self, data: sc.DataArray):
        self._data = data

    def __write_to_nexus_group__(self, group: snx.NXobject):  # required
        group.attrs['axes'] = self._data.dims  # NeXus way of defining dim labels
        group['mysignal'] = self._data.data

Note that above we use a custom "signal" name and do not to set the "signal" attribute on the group and as such deviate from the NeXus specification.
We can then write our data using:

In [ ]:
mydata = sc.DataArray(sc.arange('x', 5, unit='s'))

with snx.File('test.nxs', 'w') as f:
    f['data'] = MyData(mydata)

## Reading files

### Overview

For some application definitions &mdash; or classes within application definitions &mdash; the default ScippNexus mechanisms for reading are sufficient.
This is the case when the application definition follows the NeXus standard and, e.g., introduces no new attributes.

In other cases we require customization of how ScippNexus reads class contents.
On the lowest level, this is handled using *strategies* that can be passed to `scippnexus.NXobject` and its subclasses such as `scippnexus.NXdata`.
Handling of strategies is usually encapsulated by a *definition* that can be passed to `scippnexus.File`.

As an example, consider the following simple strategy for loading data with a custom signal name, which the file failed to specify.
We also subclass `scippnexus.ApplicationDefinition`:

In [ ]:
class MyDataStrategy(snx.NXdataStrategy):
    @staticmethod
    def signal(group: snx.NXobject) -> str:
        return 'mysignal'


class MyDefinition(snx.ApplicationDefinition):
    def make_strategy(self, group: snx.NXobject):
        if group.nx_class == snx.NXdata:
            return MyDataStrategy

We can then load our file (created above in [Writing files](#Writing-files)) by passing an instance of our custom definition to `scippnexus.File`:

In [ ]:
with snx.File('test.nxs', 'r', definition=MyDefinition()) as f:
    loaded = f['data'][...]
loaded

ScippNexus does currently not ship with a library of application definitions.
Custom definitions can be provided by a user as outlined above.
The *definition* passed to ScippNexus serves as a factory for strategies &mdash; in the above example the only custom definition is the one used for `NXdata`.
For a given group in a NeXus tree, ScippNexus calls `definition.make_strategy(group)` to request a strategy for the group.
This may return `None` and ScippNexus will then use its default strategy for loading the group.

Strategies provide customization points, e.g., for how ScippNexus can find required information in the HDF5 group, and how contents are mapped to aspects of the returned data (typically a `scipp.DataArray`).
For example, the default `scippnexus.NXdataStrategy` provides a static `axes` method that returns the name of the axes in the group, i.e., the dimension labels.
Users can either subclass `NXdataStrategy`, or provide a class with equivalent interfaces.
A full list of available base strategies can be found in [Strategies](classes.rst#Strategies).

<div class="alert alert-info">
    <b>Note:</b>

Support for strategies is very new and the number of customization points is currently very limited.
If you encounter the need for more customization, please [open an issue](https://github.com/scipp/scippnexus/issues/new) so we can consider how to extent the base strategy.

</div>

### Using strategies for filtering

The application-definition mechanism can be used for filtering or selecting which children from a group should be loaded.
For example, we may wish to exclude certain NeXus classes from loading.
We define a strategy as follows:

In [ ]:
import scippnexus as snx


class FilterStrategy(snx.NXobjectStrategy):
    @staticmethod
    def include_child(group: snx.NXobject) -> bool:
        """
        Skip all NXevent_data, the NXinstrument, and the group named "DASlogs".
        """
        skip_classes = (snx.NXevent_data, snx.NXinstrument)
        if isinstance(group, skip_classes):
            return False
        return not group.name.endswith('DASlogs')

We can use this strategy with a custom application definition as follows:

In [ ]:
from scippnexus import data

filename = data.get_path('PG3_4844_event.nxs')
definition = snx.make_definition({snx.NXentry: FilterStrategy})
f = snx.File(filename, definition=definition)
f['entry'][...]

Above we used the helper [snx.make_definition](../generated/functions/make_definition.rst) which can be used instead of manually subclassing `ApplicationDefinition` in such simple cases.